## Imports:

In [1]:
import scipy.io
import os
import pandas
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.pylab as plt_lab
import matplotlib.gridspec as gridspec
import obspy
import pandas as pd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [2]:
import warnings
warnings.filterwarnings('ignore')

## Load data:
We are going to use this data to plot spectograms of SCG signals. 

In [3]:
path = "/Users/ecem/Desktop/phonocardiogram/data/TV/present"

In [4]:
waves = np.load(path + "/present_TV_waves_10sec.npy", allow_pickle= True)


In [5]:
print(waves.shape)


(1205, 40000)


## Zero Crossing Rate:
Compute the zero-crossing rate of an audio time series.
The zero-crossing rate (ZCR) is the rate at which a signal changes from positive to zero to negative or from negative to zero to positive.

In [6]:
import librosa as lbs

In [7]:
lbs.feature.zero_crossing_rate(waves[7],frame_length= 8000, hop_length = 12000)

array([[0.014375, 0.030375, 0.03925 , 0.03725 ]])

In [8]:
lbs.feature.zero_crossing_rate(waves[7], frame_length= 8000, hop_length = 12000).shape

(1, 4)

In [9]:
def zero_crossing(array, frame_length= 2048, hl = 512):
    zero_crossing_rate = np.ones(shape = (array.shape[0], 5)) #buradaki ikinci sayı hop lenght e göre değişmeli 

    for i in range(array.shape[0]):
        zero_crossing_rate[i][0:4] = lbs.feature.zero_crossing_rate(array[i],  frame_length = frame_length,
                                                                    hop_length = hl)
        zero_crossing_rate[i][4] = sum(lbs.zero_crossings(array[i]))
    return zero_crossing_rate

        

In [10]:
zcrs = zero_crossing(waves, 8000, hl = 12000) #çok uzun sürüyor, ya datayı kısaltmalıyız ya da hop lenght artmalı

In [11]:
type(zcrs[0])

numpy.ndarray

In [12]:
np.save(path + "/present_TV_zcrs_array", zcrs, allow_pickle= True)

In [13]:
columns_zcr = []
for i in range(zcrs.shape[1]-1):
    columns_zcr.append("zcrs{}".format(i))

    
columns_zcr.append("#ofcrossing")

print(columns_zcr)

['zcrs0', 'zcrs1', 'zcrs2', 'zcrs3', '#ofcrossing']


In [14]:
zcrs[7]

array([1.4375e-02, 3.0375e-02, 3.9250e-02, 3.7250e-02, 1.3410e+03])

In [15]:
def dataframe_creator(array, df = pd.DataFrame(), l = 7, columns = None):
    values = []

    for i in range(array.shape[0]):
        llist = list(array[i][0:l-1].tolist())
        llist.append(array[i][l-1])
        
        values.append(llist)
        medium = pd.DataFrame([values[i]], columns = columns)
        
        
        df = pd.concat([df, medium], axis = 0)
    return df 

In [16]:
df = dataframe_creator(zcrs, l =5,  columns = columns_zcr)

df.reset_index(drop = True, inplace = True)

In [17]:
df.head()

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing
0,0.014375,0.045625,0.047750,0.047875,1775.0
1,0.018250,0.039250,0.049000,0.051250,1784.0
2,0.019625,0.038125,0.039375,0.039000,1647.0
3,0.025125,0.041000,0.035000,0.031875,1590.0
4,0.017625,0.043750,0.034375,0.035375,1598.0


In [18]:
df.to_csv(path + '/present_TV_zcrs.csv')

## with Özgün Ozan Nacitarhan code

In [19]:
def zcr(array):
    zcr = np.ones(shape = (array.shape[0],1))
    for i in range(array.shape[0]):
        data_zeros = np.zeros(len(array[i]))
        data_zeros[1:] = array[i][:len(array[i]) - 1]
        zcr[i][0] = (1 / (2 * len(array[i]))) * np.sum(np.abs(np.sign(array[i]) - np.sign(data_zeros)))
        
    return zcr


In [20]:
zcrs_özgün = zcr(waves)

In [21]:
zcrs_özgün.shape

(1205, 1)

In [22]:
zcrs_özgün[0]

array([0.0443625])

In [23]:
np.save(path + "/present_TV_zcrs_özgün", zcrs_özgün, allow_pickle= True)

### Chromogram:
We can use Chroma feature visualization to know how dominant the characteristics of a certain pitch {C, C♯, D, D♯, E, F, F♯, G, G♯, A, A♯, B} is present in the sampled frame.

Compute a chromagram from a waveform or power spectrogram.
It returns Normalized energy for each chroma bin at each frame.

Hop size should refer to the number of samples in between successive frames. For signal analysis Hop Size should be less than the frame size, so that frames overlap.

In [24]:
import warnings
warnings.filterwarnings('ignore')

In [25]:
import librosa.display

In [26]:
lbs.feature.chroma_stft(waves[0], sr = 4000, hop_length = 12000).shape

(12, 4)

In [27]:
# we are taking time average of each pitch values.
def chromagram_average(array, sampling_rate = 256):
    pitch = np.ones(shape = (array.shape[0], 12)) 
    
    for i in range(array.shape[0]):
        for j in range(12):
            pitch[i][j] = lbs.feature.chroma_stft(np.array(array[i]), sr= sampling_rate, hop_length = 12000)[j].mean()
    return pitch

In [28]:
chro = chromagram_average(waves, sampling_rate = 4000) # bu da çok uzun sürüyor.

In [29]:
type(chro[0])

numpy.ndarray

In [30]:
chro.shape

(1205, 12)

In [31]:
np.save(path + "/present_TV_chromogram_array", chro, allow_pickle= True)

In [32]:
print(chro.shape)

(1205, 12)


In [33]:
lst = ["C", "C♯", "D", "D♯", "E", "F", "F♯", "G", "G♯", "A", "A♯", "B"]

In [34]:
col_chro = []
for i in range(chro.shape[1]):
    col_chro.append(lst[i])
    
print(col_chro)
print(len(col_chro))

['C', 'C♯', 'D', 'D♯', 'E', 'F', 'F♯', 'G', 'G♯', 'A', 'A♯', 'B']
12


In [35]:
ch_df = dataframe_creator(chro,
                         df =pd.DataFrame(columns = col_chro), columns = col_chro, l = 12)

ch_df.reset_index(drop = True, inplace = True)
ch_df

,C,C♯,D,D♯,E,F,F♯,G,G♯,A,A♯,B
0,0.632728,0.582556,0.603655,0.691997,0.506335,0.496066,0.553746,0.565339,0.520591,0.54311,0.587228,0.465919
1,0.758642,0.864754,0.811397,0.733465,0.694984,0.593114,0.590263,0.54018,0.517904,0.802246,0.897468,0.905409
2,0.636057,0.625512,0.647747,0.699375,0.545241,0.557489,0.403775,0.325962,0.417445,0.533893,0.668166,0.567049
3,0.628635,0.557179,0.529808,0.537103,0.520591,0.644424,0.603671,0.515941,0.483744,0.619612,0.640899,0.652614
4,0.3831,0.359934,0.39486,0.351761,0.431666,0.59696,0.580548,0.597486,0.557329,0.626974,0.62526,0.583547
...,...,...,...,...,...,...,...,...,...,...,...,...
1200,0.761378,0.58194,0.589731,0.564518,0.530238,0.632826,0.738431,0.799368,0.852685,0.710811,0.692615,0.862163
1201,0.630042,0.788746,0.75657,0.753607,0.664939,0.692472,0.805693,0.668857,0.633761,0.697793,0.588426,0.541017
1202,0.550638,0.775268,0.765337,0.790918,0.585392,0.43467,0.335148,0.359834,0.445198,0.50422,0.42853,0.471036
1203,0.636011,0.748156,0.76196,0.681554,0.668069,0.752171,0.865487,0.817557,0.768924,0.7034,0.654348,0.604923


In [36]:
ch_df.to_csv(path + "/present_TV_chromogram.csv")

## Tempogram

Compute the tempogram: local autocorrelation of the onset strength envelope.

# kendime not tempogram değerlerini spektogram değerleri gibi kaydet.

In [37]:
def tempo_average(array): # 192,79
    means= []
    for i in range(array.shape[0]): #192
        means.append(array[i].mean())
    return np.array(means) # 192
        

In [38]:
def tempogram(array = 0, hop_lenght = 12000, sampling_rate = 256):
    
    oenv = librosa.onset.onset_strength(y=np.array(array), 
                                            sr=sampling_rate, 
                                        lag =3)

    times = librosa.times_like(oenv, sr=sampling_rate, hop_length=12000)

    tempogram = librosa.feature.tempogram(onset_envelope = oenv, sr = sampling_rate,
                                              hop_length = hop_lenght, win_length = 192 )
    # Estimate the global tempo for display purposes
    #tempo = librosa.beat.tempo(onset_envelope=oenv, sr= sampling_rate,
                                   #hop_length=hop_length)[0]
    return tempogram

In [39]:
tempogram(waves[0]).shape

(192, 79)

In [40]:
tempogram(waves[0])[0]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

In [41]:
tempogram(waves[0])[1]

array([0.76912238, 0.76964087, 0.77014655, 0.77063979, 0.77112095,
       0.77159038, 0.77204843, 0.77249542, 0.7729317 , 0.77335758,
       0.7737734 , 0.77417946, 0.77457607, 0.77496353, 0.77534216,
       0.77571223, 0.77607405, 0.77642789, 0.77677405, 0.77711279,
       0.77744438, 0.77776911, 0.77808724, 0.77839903, 0.77870475,
       0.77900465, 0.77929898, 0.77958802, 0.779872  , 0.78015119,
       0.78042583, 0.78069617, 0.78096246, 0.78122497, 0.78148393,
       0.7817396 , 0.78199224, 0.7822421 , 0.78248944, 0.78273452,
       0.7829776 , 0.78321896, 0.78345887, 0.7836976 , 0.78393544,
       0.78417268, 0.78440962, 0.78464656, 0.78488381, 0.78512169,
       0.78536055, 0.78560071, 0.78584254, 0.7860864 , 0.78633268,
       0.78658176, 0.78683405, 0.78709   , 0.78735003, 0.78761462,
       0.78788426, 0.78815944, 0.7884407 , 0.7887286 , 0.78902372,
       0.78932667, 0.78963809, 0.78995866, 0.79028909, 0.79063011,
       0.79098252, 0.79134714, 0.79172483, 0.79211651, 0.79252

In [42]:
tempo_average(tempogram(waves[0]))

array([ 1.00000000e+00,  7.82314233e-01,  4.43080141e-01,  2.69458564e-01,
        2.60348607e-01,  2.72531839e-01,  2.79538468e-01,  2.73401601e-01,
        2.51887344e-01,  2.84713155e-01,  4.25587903e-01,  6.23928574e-01,
        7.09609367e-01,  6.25507524e-01,  4.32024751e-01,  2.78093031e-01,
        2.21082391e-01,  2.35006751e-01,  2.47217670e-01,  2.49768980e-01,
        2.35908171e-01,  2.24142581e-01,  3.13273601e-01,  5.24804134e-01,
        6.72220964e-01,  5.64154998e-01,  3.49821509e-01,  2.20842442e-01,
        1.92250807e-01,  1.83570667e-01,  1.89928875e-01,  1.92464145e-01,
        1.77605428e-01,  1.85066321e-01,  2.53034411e-01,  3.46508151e-01,
        3.77849358e-01,  3.34281290e-01,  2.49771728e-01,  1.77438175e-01,
        1.30133949e-01,  1.24878609e-01,  1.25698068e-01,  1.28299268e-01,
        1.27526037e-01,  1.22363029e-01,  1.47860372e-01,  2.17112542e-01,
        2.73866609e-01,  2.41856049e-01,  1.64897217e-01,  1.08925193e-01,
        9.02281584e-02,  

# tempogram(waves[0])[0]

In [43]:
def temp_values(array):
    final = np.ones(shape = (array.shape[0], 192))
    for i in range(array.shape[0]): #8bin civarı
        
        _x = tempogram(array[i]) #1,4000   #192,79
        #print(_x.shape)
        final[i] = tempo_average(_x) #192
        
        
    return final

In [44]:
temp = temp_values(waves) #çok uzun sürüyor 


In [45]:
type(temp)

numpy.ndarray

In [46]:
np.save(path + "/present_TV_tempogram_array", temp, allow_pickle= True)

In [47]:
temp.shape

(1205, 192)

In [48]:
columns_t = []
for i in range(temp.shape[1]):
    columns_t.append("tempogram_{}".format(i))
print(columns_t)

['tempogram_0', 'tempogram_1', 'tempogram_2', 'tempogram_3', 'tempogram_4', 'tempogram_5', 'tempogram_6', 'tempogram_7', 'tempogram_8', 'tempogram_9', 'tempogram_10', 'tempogram_11', 'tempogram_12', 'tempogram_13', 'tempogram_14', 'tempogram_15', 'tempogram_16', 'tempogram_17', 'tempogram_18', 'tempogram_19', 'tempogram_20', 'tempogram_21', 'tempogram_22', 'tempogram_23', 'tempogram_24', 'tempogram_25', 'tempogram_26', 'tempogram_27', 'tempogram_28', 'tempogram_29', 'tempogram_30', 'tempogram_31', 'tempogram_32', 'tempogram_33', 'tempogram_34', 'tempogram_35', 'tempogram_36', 'tempogram_37', 'tempogram_38', 'tempogram_39', 'tempogram_40', 'tempogram_41', 'tempogram_42', 'tempogram_43', 'tempogram_44', 'tempogram_45', 'tempogram_46', 'tempogram_47', 'tempogram_48', 'tempogram_49', 'tempogram_50', 'tempogram_51', 'tempogram_52', 'tempogram_53', 'tempogram_54', 'tempogram_55', 'tempogram_56', 'tempogram_57', 'tempogram_58', 'tempogram_59', 'tempogram_60', 'tempogram_61', 'tempogram_62', '

In [49]:
len(columns_t)

192

In [50]:
df_t = dataframe_creator(temp,
                         df =pd.DataFrame(columns = columns_t), columns = columns_t, l = 192)

df_t.reset_index(drop = True, inplace = True)
df_t

,tempogram_0,tempogram_1,tempogram_2,tempogram_3,tempogram_4,tempogram_5,tempogram_6,tempogram_7,tempogram_8,tempogram_9,...,tempogram_182,tempogram_183,tempogram_184,tempogram_185,tempogram_186,tempogram_187,tempogram_188,tempogram_189,tempogram_190,tempogram_191
0,1.0,0.782314,0.44308,0.269459,0.260349,0.272532,0.279538,0.273402,0.251887,0.284713,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.774253,0.440555,0.284891,0.311018,0.333963,0.349042,0.326007,0.293324,0.278827,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2,1.0,0.867157,0.684033,0.598988,0.592096,0.602322,0.616906,0.613713,0.590528,0.573863,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,1.0,0.705319,0.316058,0.135829,0.303616,0.531777,0.579377,0.370918,0.21552,0.186584,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.738496,0.348824,0.127286,0.071349,0.1194,0.212045,0.306395,0.391691,0.470576,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,1.0,0.781321,0.424697,0.198154,0.15576,0.172316,0.181592,0.198851,0.235288,0.271238,...,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0
1201,1.0,0.777808,0.40789,0.188958,0.158389,0.181104,0.194287,0.189864,0.220658,0.299185,...,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1202,1.0,0.743368,0.386358,0.181704,0.192026,0.322006,0.377815,0.32525,0.317242,0.369902,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1203,1.0,0.902151,0.734999,0.633144,0.600471,0.588014,0.565444,0.544215,0.547051,0.555489,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


In [51]:
df_t['tempogram_188'][10]

8.533221099706138e-18

In [52]:
df_t.to_csv(path + '/present_TV_tempogram.csv')

In [53]:
df_t.isin([0]).sum(axis=0)

tempogram_0      0
tempogram_1      0
tempogram_2      0
tempogram_3      0
tempogram_4      0
                ..
tempogram_187    0
tempogram_188    0
tempogram_189    0
tempogram_190    0
tempogram_191    0
Length: 192, dtype: int64

In [54]:
final_df = pd.concat([df, ch_df], axis =1)
final_df

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing,C,C♯,D,D♯,E,F,F♯,G,G♯,A,A♯,B
0,0.014375,0.045625,0.047750,0.047875,1775.0,0.632728,0.582556,0.603655,0.691997,0.506335,0.496066,0.553746,0.565339,0.520591,0.54311,0.587228,0.465919
1,0.018250,0.039250,0.049000,0.051250,1784.0,0.758642,0.864754,0.811397,0.733465,0.694984,0.593114,0.590263,0.54018,0.517904,0.802246,0.897468,0.905409
2,0.019625,0.038125,0.039375,0.039000,1647.0,0.636057,0.625512,0.647747,0.699375,0.545241,0.557489,0.403775,0.325962,0.417445,0.533893,0.668166,0.567049
3,0.025125,0.041000,0.035000,0.031875,1590.0,0.628635,0.557179,0.529808,0.537103,0.520591,0.644424,0.603671,0.515941,0.483744,0.619612,0.640899,0.652614
4,0.017625,0.043750,0.034375,0.035375,1598.0,0.3831,0.359934,0.39486,0.351761,0.431666,0.59696,0.580548,0.597486,0.557329,0.626974,0.62526,0.583547
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,0.005375,0.009125,0.008625,0.009375,382.0,0.761378,0.58194,0.589731,0.564518,0.530238,0.632826,0.738431,0.799368,0.852685,0.710811,0.692615,0.862163
1201,0.006000,0.008500,0.009250,0.009625,388.0,0.630042,0.788746,0.75657,0.753607,0.664939,0.692472,0.805693,0.668857,0.633761,0.697793,0.588426,0.541017
1202,0.004500,0.009375,0.007500,0.008125,358.0,0.550638,0.775268,0.765337,0.790918,0.585392,0.43467,0.335148,0.359834,0.445198,0.50422,0.42853,0.471036
1203,0.004000,0.009000,0.010000,0.012750,456.0,0.636011,0.748156,0.76196,0.681554,0.668069,0.752171,0.865487,0.817557,0.768924,0.7034,0.654348,0.604923


In [55]:
final_df = pd.concat([final_df, df_t], axis=1)
final_df

,zcrs0,zcrs1,zcrs2,zcrs3,#ofcrossing,C,C♯,D,D♯,E,...,tempogram_182,tempogram_183,tempogram_184,tempogram_185,tempogram_186,tempogram_187,tempogram_188,tempogram_189,tempogram_190,tempogram_191
0,0.014375,0.045625,0.047750,0.047875,1775.0,0.632728,0.582556,0.603655,0.691997,0.506335,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.018250,0.039250,0.049000,0.051250,1784.0,0.758642,0.864754,0.811397,0.733465,0.694984,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
2,0.019625,0.038125,0.039375,0.039000,1647.0,0.636057,0.625512,0.647747,0.699375,0.545241,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
3,0.025125,0.041000,0.035000,0.031875,1590.0,0.628635,0.557179,0.529808,0.537103,0.520591,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.017625,0.043750,0.034375,0.035375,1598.0,0.3831,0.359934,0.39486,0.351761,0.431666,...,-0.0,0.0,0.0,-0.0,0.0,-0.0,-0.0,-0.0,-0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1200,0.005375,0.009125,0.008625,0.009375,382.0,0.761378,0.58194,0.589731,0.564518,0.530238,...,-0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,0.0,0.0,0.0
1201,0.006000,0.008500,0.009250,0.009625,388.0,0.630042,0.788746,0.75657,0.753607,0.664939,...,-0.0,-0.0,-0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1202,0.004500,0.009375,0.007500,0.008125,358.0,0.550638,0.775268,0.765337,0.790918,0.585392,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0
1203,0.004000,0.009000,0.010000,0.012750,456.0,0.636011,0.748156,0.76196,0.681554,0.668069,...,-0.0,0.0,0.0,0.0,0.0,-0.0,0.0,0.0,0.0,0.0


In [56]:
final_df.to_csv(path + '/present_TV_final_sp.csv')